# Label-Free Concept Bottleneck Model (LF-CBM) for Brain Scan Classification

This notebook implements an explainable AI approach using BioMedCLIP to classify brain scans based on clinical concepts rather than raw pixels.

**Pipeline:**
1. Load BioMedCLIP pre-trained model
2. Define clinical concept bank
3. Extract image-text similarity scores
4. Train logistic regression on concept scores
5. Analyze concept importance for explainability

## Step 1: Install Required Packages

First, let's install the necessary packages for BioMedCLIP.

In [ ]:
# Install required packages
!pip install transformers torch torchvision pillow scikit-learn open_clip_torch tqdm

## Step 2: Import Libraries and Setup Device

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer, AutoImageProcessor
from PIL import Image
import numpy as np
import os
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## Step 3: Load BioMedCLIP Model

Load the pre-trained BioMedCLIP model from Microsoft.

In [ ]:
print("Loading BioMedCLIP model...")
model_name = "microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224"

# Load model, tokenizer, and image processor
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
image_processor = AutoImageProcessor.from_pretrained(model_name, trust_remote_code=True)

# Move model to device
model = model.to(device)
model.eval()

print(f"✓ BioMedCLIP model loaded successfully on {device}")

## Step 4: Define Clinical Concept Bank

Define the clinical concepts for each condition that will serve as our interpretable features.

### Dataset Visual Analysis Results

Before defining concepts, let's review the visual characteristics of your dataset:

**Hemorrhagic Images:**
- Primary sequences: SWI (77%), DWI (14%), GRE (8%)
- Average brightness: 0.258 (moderate)
- Key features: Dark signals, blooming artifacts, well-defined boundaries

**Ischemic Images:**
- Primary sequences: Mixed/DWI-based
- Average brightness: 0.194 (darkest class)
- Key features: Territorial patterns, diffusion restriction, wedge-shaped lesions

**Tumor Images:**
- Various MRI sequences
- Average brightness: 0.301 (brightest class)
- Key features: Mass effect, irregular margins, heterogeneous signals

These observations inform our concept selection below.

In [ ]:
# Define clinical concepts for each class
# These concepts are specifically tailored to this dataset based on visual analysis
# Dataset characteristics:
# - Hemorrhagic: 77% SWI, 14% DWI, 8% GRE | Brightness: 0.258, Contrast: 0.313
# - Ischemic: Mixed sequences (73% unlabeled), 17% DWI | Brightness: 0.194 (darkest), Contrast: 0.225
# - Tumor: Various contrasts | Brightness: 0.301 (brightest), Edge Density: 0.0414 (highest)

concept_bank = {
    "Tumor": [
        "space-occupying lesion",
        "mass effect present",
        "irregular lesion margins",
        "vasogenic edema pattern",
        "central hypointensity",
        "heterogeneous signal intensity",
        "fingerlike edema extension",
        "structural displacement",
        "necrotic center"
    ],
    "Hemorrhagic": [
        "dark signal on susceptibility weighted imaging",
        "blooming artifact present",
        "focal hypointense region",
        "blood products signal intensity",
        "susceptibility artifact",
        "well-circumscribed hemorrhagic lesion",
        "acute bleeding pattern",
        "hypointense on SWI"
    ],
    "Ischemic": [
        "bright signal on diffusion weighted imaging",
        "territorial distribution pattern",
        "vascular territory involvement",
        "diffusion restriction present",
        "wedge-shaped lesion",
        "cortical gray matter involvement",
        "hypointense region on apparent diffusion coefficient",
        "acute infarct pattern"
    ],
    "Normal": [
        "normal gray matter signal",
        "normal white matter signal",
        "symmetric brain structures",
        "no abnormal signal intensity",
        "preserved brain anatomy"
    ]
}

# Flatten all concepts into a single list
all_concepts = []
concept_to_class = {}
for class_name, concepts in concept_bank.items():
    for concept in concepts:
        all_concepts.append(concept)
        concept_to_class[concept] = class_name

print(f"Total concepts: {len(all_concepts)}")
print("\nDataset-Specific Concept Bank:")
print("(Based on analysis of your processed images)\n")
for class_name, concepts in concept_bank.items():
    print(f"\n{class_name} ({len(concepts)} concepts):")
    for concept in concepts:
        print(f"  - {concept}")

## Step 5: Prepare Text Embeddings

Pre-compute text embeddings for all concepts to speed up the feature extraction process.

In [ ]:
print("Computing text embeddings for all concepts...")

# Tokenize all concepts
text_inputs = tokenizer(all_concepts, padding=True, truncation=True, return_tensors="pt")
text_inputs = {k: v.to(device) for k, v in text_inputs.items()}

# Get text embeddings
with torch.no_grad():
    text_embeddings = model.get_text_features(**text_inputs)
    text_embeddings = F.normalize(text_embeddings, dim=-1)

print(f"✓ Text embeddings shape: {text_embeddings.shape}")
print(f"  ({text_embeddings.shape[0]} concepts × {text_embeddings.shape[1]} dimensions)")

## Step 6: Load Dataset

Load all processed images from the dataset directories.

In [ ]:
# Define dataset path
data_dir = Path("../content/processed_data")

# Class mapping
class_names = ["Hemorrhagic", "Ischemic", "Tumor"]
class_to_idx = {name: idx for idx, name in enumerate(class_names)}

# Load image paths and labels
image_paths = []
labels = []

for class_name in class_names:
    class_dir = data_dir / class_name
    if not class_dir.exists():
        print(f"Warning: Directory {class_dir} not found!")
        continue
    
    # Get all image files
    image_files = list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png"))
    
    for img_path in image_files:
        image_paths.append(str(img_path))
        labels.append(class_to_idx[class_name])
    
    print(f"{class_name}: {len(image_files)} images")

print(f"\nTotal images: {len(image_paths)}")
print(f"Labels distribution: {np.bincount(labels)}")

## Step 7: Extract Image-Text Similarity Features

For each image, compute similarity scores with all clinical concepts using BioMedCLIP.

In [ ]:
def extract_concept_features(image_path, text_embeddings, model, image_processor, device):
    """
    Extract concept-based features for a single image.
    Returns similarity scores between the image and all concepts.
    """
    # Load and preprocess image
    image = Image.open(image_path).convert("RGB")
    inputs = image_processor(images=image, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Get image embedding
    with torch.no_grad():
        image_embedding = model.get_image_features(**inputs)
        image_embedding = F.normalize(image_embedding, dim=-1)
    
    # Compute similarity with all concepts (cosine similarity via dot product)
    similarities = (image_embedding @ text_embeddings.T).squeeze(0)
    
    return similarities.cpu().numpy()

# Extract features for all images
print("Extracting concept features from images...")
print("This may take a few minutes depending on dataset size...\n")

X_features = []
y_labels = []

for img_path, label in tqdm(zip(image_paths, labels), total=len(image_paths), desc="Processing images"):
    try:
        features = extract_concept_features(img_path, text_embeddings, model, image_processor, device)
        X_features.append(features)
        y_labels.append(label)
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        continue

# Convert to numpy arrays
X = np.array(X_features)
y = np.array(y_labels)

print(f"\n✓ Feature extraction complete!")
print(f"Feature matrix shape: {X.shape}")
print(f"  ({X.shape[0]} images × {X.shape[1]} concept scores)")
print(f"Labels shape: {y.shape}")

## Step 8: Train-Test Split

Split the concept features into training and testing sets.

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} images")
print(f"Testing set: {X_test.shape[0]} images")
print(f"\nTraining labels distribution: {np.bincount(y_train)}")
print(f"Testing labels distribution: {np.bincount(y_test)}")

## Step 9: Train Concept Bottleneck Model

Train a logistic regression classifier on the concept scores.

In [ ]:
print("Training Logistic Regression on concept scores...\n")

# Train logistic regression classifier
clf = LogisticRegression(
    max_iter=1000,
    random_state=42,
    multi_class='multinomial',
    solver='lbfgs',
    C=1.0
)

clf.fit(X_train, y_train)

# Make predictions
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Calculate accuracies
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"✓ Training complete!")
print(f"\n{'='*50}")
print(f"MODEL PERFORMANCE")
print(f"{'='*50}")
print(f"Training Accuracy: {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"Testing Accuracy:  {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"{'='*50}")

## Step 10: Detailed Classification Report

View detailed metrics for each class.

In [ ]:
print("\nClassification Report (Test Set):")
print("="*70)
print(classification_report(y_test, y_test_pred, target_names=class_names))

# Confusion Matrix
cm = confusion_matrix(y_test, y_test_pred)
print("\nConfusion Matrix:")
print(cm)

## Step 11: Visualize Confusion Matrix

Create a visual representation of the confusion matrix.

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - LF-CBM Model', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

# Normalized confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Percentage'})
plt.title('Normalized Confusion Matrix - LF-CBM Model', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

## Step 12: Explainability - Concept Importance Analysis

Analyze which concepts are most important for each class prediction.

In [ ]:
# Extract weights from logistic regression
weights = clf.coef_  # Shape: (n_classes, n_concepts)
print(f"Model weights shape: {weights.shape}")
print(f"  ({weights.shape[0]} classes × {weights.shape[1]} concepts)\n")

# Create DataFrame for better visualization
concept_importance = pd.DataFrame(
    weights.T,
    columns=class_names,
    index=all_concepts
)

print("="*80)
print("CONCEPT WEIGHTS FOR EACH CLASS")
print("="*80)
print("\nPositive weights indicate concepts that support the class prediction.")
print("Negative weights indicate concepts that oppose the class prediction.\n")

for class_name in class_names:
    print(f"\n{'='*80}")
    print(f"TOP CONCEPTS FOR: {class_name.upper()}")
    print(f"{'='*80}")
    
    # Sort concepts by absolute weight
    sorted_concepts = concept_importance[class_name].abs().sort_values(ascending=False)
    
    print(f"\nTop 10 Most Important Concepts (by absolute weight):\n")
    for i, (concept, _) in enumerate(sorted_concepts.head(10).items(), 1):
        weight = concept_importance.loc[concept, class_name]
        impact = "Supporting" if weight > 0 else "Opposing"
        print(f"{i:2d}. {concept:35s} | Weight: {weight:+7.4f} | {impact}")
    
    print(f"\n{'-'*80}")
    print("Top 5 Supporting Concepts (highest positive weights):\n")
    top_positive = concept_importance[class_name].sort_values(ascending=False).head(5)
    for i, (concept, weight) in enumerate(top_positive.items(), 1):
        print(f"{i}. {concept:35s} | Weight: {weight:+7.4f}")
    
    print(f"\n{'-'*80}")
    print("Top 5 Opposing Concepts (most negative weights):\n")
    top_negative = concept_importance[class_name].sort_values(ascending=True).head(5)
    for i, (concept, weight) in enumerate(top_negative.items(), 1):
        print(f"{i}. {concept:35s} | Weight: {weight:+7.4f}")

## Step 13: Visualize Concept Importance

Create heatmaps to visualize which concepts are important for each class.

In [ ]:
# Heatmap of all concept weights
plt.figure(figsize=(12, 10))
sns.heatmap(concept_importance, cmap='RdBu_r', center=0, 
            annot=True, fmt='.3f', linewidths=0.5,
            cbar_kws={'label': 'Weight'})
plt.title('Concept Importance Heatmap\n(Weights from Logistic Regression)', 
          fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Predicted Class', fontsize=12)
plt.ylabel('Clinical Concepts', fontsize=12)
plt.tight_layout()
plt.show()

# Bar plot for each class
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, class_name in enumerate(class_names):
    ax = axes[idx]
    
    # Get top 10 concepts by absolute weight
    top_concepts = concept_importance[class_name].abs().sort_values(ascending=False).head(10)
    top_concepts_signed = concept_importance.loc[top_concepts.index, class_name]
    
    # Create color based on sign
    colors = ['green' if w > 0 else 'red' for w in top_concepts_signed]
    
    # Plot
    top_concepts_signed.sort_values().plot(kind='barh', ax=ax, color=colors, alpha=0.7)
    ax.set_title(f'Top 10 Concepts for {class_name}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Weight', fontsize=10)
    ax.set_ylabel('Concept', fontsize=10)
    ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
    ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## Step 14: Example Prediction with Explanation

Demonstrate how to make a prediction with concept-based explanation.

In [ ]:
def explain_prediction(image_path, model, image_processor, text_embeddings, 
                      clf, all_concepts, class_names, device, top_k=5):
    """
    Make a prediction and explain it using concept scores.
    """
    # Extract concept features
    features = extract_concept_features(image_path, text_embeddings, model, 
                                       image_processor, device)
    
    # Make prediction
    prediction = clf.predict(features.reshape(1, -1))[0]
    probabilities = clf.predict_proba(features.reshape(1, -1))[0]
    predicted_class = class_names[prediction]
    
    print("="*80)
    print(f"PREDICTION EXPLANATION FOR: {image_path}")
    print("="*80)
    print(f"\n🔍 Predicted Class: {predicted_class}")
    print(f"\nClass Probabilities:")
    for i, class_name in enumerate(class_names):
        prob = probabilities[i]
        bar = '█' * int(prob * 50)
        print(f"  {class_name:15s}: {prob:.4f} ({prob*100:.2f}%) {bar}")
    
    print(f"\n{'='*80}")
    print(f"CONCEPT ACTIVATION SCORES")
    print(f"{'='*80}")
    print("\nHighest Activated Concepts (indicating presence):\n")
    
    # Get top activated concepts
    concept_scores = pd.Series(features, index=all_concepts)
    top_activated = concept_scores.sort_values(ascending=False).head(top_k)
    
    for i, (concept, score) in enumerate(top_activated.items(), 1):
        # Get weight for predicted class
        concept_idx = all_concepts.index(concept)
        weight = clf.coef_[prediction, concept_idx]
        contribution = score * weight
        
        print(f"{i}. {concept:35s}")
        print(f"   Similarity Score: {score:.4f}")
        print(f"   Weight for {predicted_class}: {weight:+.4f}")
        print(f"   Contribution: {contribution:+.4f}")
        print()
    
    print(f"{'='*80}")
    print(f"INTERPRETATION")
    print(f"{'='*80}")
    print(f"\nThe model predicted '{predicted_class}' based on the following reasoning:")
    print(f"\nTop supporting evidence:")
    
    # Find concepts with high score AND high positive weight for predicted class
    contributions = []
    for i, concept in enumerate(all_concepts):
        score = features[i]
        weight = clf.coef_[prediction, i]
        contrib = score * weight
        contributions.append((concept, score, weight, contrib))
    
    # Sort by contribution
    contributions.sort(key=lambda x: x[3], reverse=True)
    
    for i, (concept, score, weight, contrib) in enumerate(contributions[:3], 1):
        if contrib > 0:
            print(f"{i}. Detected '{concept}' (similarity: {score:.3f}, weight: {weight:+.3f})")
    
    return prediction, probabilities, features

# Test on a random sample from test set
test_idx = np.random.randint(0, len(X_test))
test_image_path = image_paths[test_idx]
true_label = class_names[y[test_idx]]

print(f"Testing on image from: {test_image_path}")
print(f"True label: {true_label}\n")

prediction, probs, features = explain_prediction(
    test_image_path, model, image_processor, text_embeddings,
    clf, all_concepts, class_names, device, top_k=5
)

## Step 15: Save the Model

Save the trained model and concept bank for future use.

In [ ]:
import pickle
import json

# Create models directory if it doesn't exist
os.makedirs("../models", exist_ok=True)

# Save the logistic regression model
model_path = "../models/lfcbm_classifier.pkl"
with open(model_path, 'wb') as f:
    pickle.dump(clf, f)
print(f"✓ Saved classifier to: {model_path}")

# Save the concept bank and metadata
metadata = {
    "concept_bank": concept_bank,
    "all_concepts": all_concepts,
    "class_names": class_names,
    "class_to_idx": class_to_idx,
    "model_name": model_name,
    "test_accuracy": float(test_acc),
    "train_accuracy": float(train_acc)
}

metadata_path = "../models/lfcbm_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✓ Saved metadata to: {metadata_path}")

# Save concept importance
importance_path = "../models/lfcbm_concept_importance.csv"
concept_importance.to_csv(importance_path)
print(f"✓ Saved concept importance to: {importance_path}")

print(f"\n{'='*80}")
print("MODEL SAVED SUCCESSFULLY")
print(f"{'='*80}")
print("\nYou can now load and use this model for inference with explainability!")

## Summary

This notebook implemented a **Label-Free Concept Bottleneck Model (LF-CBM)** for explainable brain scan classification:

### Key Components:
1. **BioMedCLIP**: Pre-trained medical vision-language model
2. **Concept Bank**: Clinical concepts for Tumor, Hemorrhagic, Ischemic, and Normal brain
3. **Feature Extraction**: Image-text similarity scores as interpretable features
4. **Classification**: Logistic Regression on concept scores
5. **Explainability**: Concept weights show which clinical features drive predictions

### Benefits:
- ✅ **Interpretable**: Predictions explained by clinical concepts
- ✅ **Transparent**: See which concepts contribute to each decision
- ✅ **Medical-relevant**: Uses domain-specific concepts from BioMedCLIP
- ✅ **No manual labeling**: Label-free approach using vision-language alignment

### How to Use:
The model can now predict with explanations like:
> "Predicted **Tumor** because:
> - High 'mass effect' score (0.85)
> - High 'midline shift' score (0.78)
> - High 'edema surrounding mass' score (0.72)"

This provides clinically meaningful insights into the model's decision-making process!